In [43]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount ("/content/gdrive")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
#Se puede mejorar pasando una lista de empty symbols sacados del analisis del dataset
def iscomplete(x, y):
  if np.isnan(x) == False and np.isnan(y) == False:
    return True
  return False

In [45]:
def euclidean_distance(X, Y):
  if len(X) != len(Y):
    print("Not same size")
    return
  result = 0
  for i in range(len(X)):
    if iscomplete(X[i], Y[i]):
      result += (X[i] - Y[i])**2
  result **= 0.5
  return result

In [46]:
def manhattan_distance(X, Y):
  if len(X) != len(Y):
    print("Not same size")
    return
  result = 0
  for i in range(len(X)):
    if iscomplete(X[i], Y[i]):
      result += abs(X[i] - Y[i])
  return result

In [47]:
def pearson_correlation(X, Y):
  if len(X) != len(Y):
    print("Not same size")
    return
  sum_xy = 0
  sum_x = 0
  sum_y = 0
  sum_x2 = 0
  sum_y2 = 0
  n = 0
  for i in range(len(X)):
    if iscomplete(X[i], Y[i]):
      sum_xy += X[i]*Y[i]
      sum_x += X[i]
      sum_y += Y[i]
      sum_x2 += X[i]**2
      sum_y2 += Y[i]**2
      n += 1 
  result = sum_xy - (sum_x*sum_y)/n
  result /= (sum_x2 - (sum_x**2)/n)**0.5 *  (sum_y2 - (sum_y**2)/n)**0.5
  return result



In [56]:
def pearson_correlation_book(X, Y):
  if len(X) != len(Y):
    print("Not same size")
    return
  sum_xy = 0
  sum_x = 0
  sum_y = 0
  sum_x2 = 0
  sum_y2 = 0
  n = 0
  for i in range(len(X)):
    if iscomplete(X[i], Y[i]) == False:
      if np.isnan(X[i]) == True:
        X[i] = 0
      elif np.isnan(Y[i]) == True:
        Y[i] = 0
    sum_xy += X[i]*Y[i]
    sum_x += X[i]
    sum_y += Y[i]
    sum_x2 += X[i]**2
    sum_y2 += Y[i]**2
    n += 1 
  result = sum_xy - (sum_x*sum_y)/n
  result /= (sum_x2 - (sum_x**2)/n)**0.5 *  (sum_y2 - (sum_y**2)/n)**0.5
  return result

In [53]:
def cos_similarity(X, Y):
  if len(X) != len(Y):
    print("Not same size")
    return
  dot_xy = 0
  mod_x = 0
  mod_y = 0
  for i in range(len(X)):
    if iscomplete(X[i], Y[i]):
      dot_xy += X[i]*Y[i]
      mod_x += X[i]**2
      mod_y += Y[i]**2
  mod_x **= 0.5
  mod_y **= 0.5
  result = dot_xy/(mod_x * mod_y)
  return result

In [55]:
def cos_similarity_book(X, Y):
  if len(X) != len(Y):
    print("Not same size")
    return
  dot_xy = 0
  mod_x = 0
  mod_y = 0
  for i in range(len(X)):
    if iscomplete(X[i], Y[i]) == False:
      if np.isnan(X[i]) == True:
        X[i] = 0
      elif np.isnan(Y[i]) == True:
        Y[i] = 0
    dot_xy += X[i]*Y[i]
    mod_x += X[i]**2
    mod_y += Y[i]**2 
  mod_x **= 0.5
  mod_y **= 0.5
  result = dot_xy/(mod_x * mod_y)
  return result

In [49]:
#Data
df_basic_dataset = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/CS. Datos/datasets/basic_dataset.csv")
df_movie_ratings = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/CS. Datos/datasets/Movie_Ratings.csv")
df_basic_dataset_movies_names = df_basic_dataset.iloc[0:,0]
df_movie_ratings_movies_names = df_movie_ratings.iloc[0:,0]
df_basic_dataset = df_basic_dataset.apply(lambda x: pd.to_numeric(x, errors='coerce'))


In [50]:
#Testing
def test_dataset(x_index, y_index, dataset):
  data = dataset
  md = manhattan_distance(data[x_index], data[y_index])
  ed = euclidean_distance(data[x_index], data[y_index])
  pc = pearson_correlation(data[x_index], data[y_index])
  cs = cos_similarity(data[x_index], data[y_index])
  print(f'La distancia de manhattan entre {x_index} y {y_index} es {md}')
  print(f'La distancia euclideana entre {x_index} y {y_index} es {ed}')
  print(f'El correlativo de pearson entre {x_index} y {y_index} es {pc}')
  print(f'La similitud de coseno entre {x_index} y {y_index} es {cs}')
  print('-----------------------------------------------------------------------')

In [67]:
data = df_movie_ratings
print('cos Jessica y Jeff: ', cos_similarity(data['Jessica'], data['Jeff']))
print('pearson Matt y Josh: ', pearson_correlation(data['Matt'], data['Josh']))
print('cos Matt y Josh: ', cos_similarity(data['Matt'], data['Josh']))
print('euclidean Erin y Patrick T: ', euclidean_distance(data['Erin'], data['Patrick T']))
print('manhattan ben y Gary: ', manhattan_distance(data['ben'], data['Gary']))

cos Jessica y Jeff:  0.9741763740941048
pearson Matt y Josh:  nan
cos Matt y Josh:  0.9918365981341757
euclidean Erin y Patrick T:  5.744562646538029
manhattan ben y Gary:  21.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [68]:
test_dataset('Matt', 'Josh', data)

La distancia de manhattan entre Matt y Josh es 1.0
La distancia euclideana entre Matt y Josh es 1.0
El correlativo de pearson entre Matt y Josh es nan
La similitud de coseno entre Matt y Josh es 0.9918365981341757
-----------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
